### Para quê paralelismo?

Ao longo dos últimos anos pudemos ver uma certa estagnação na evolução da velocidade dos núcleos dos processadores ainda que hoje existam bem mais núcleos disponíveis nos processadores de hoje em relação aos de anos atrás. Por isso mesmo buscar formas de explorar os demais núcleos se tornou um desafio para as linguagens de programação e aqui é que aparecem as coroutines, como subprocessos leves que têm a liberdade de operar em diferentes núcleos do processador de forma asíncrona.

Em Julia aproveitamos este recurso através de macros mas para termos a possibilidade de aproveitar por completo o processador, precisamos inicialmente iniciar os demais processos.

In [1]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Ao contrário de outras linguagens onde para iniciarmos uma thread precisaríamos criar uma classes e adicionar o método *run()*, em Julia basta apenas utilizarmos algumas macros

In [3]:
# criando um array compartilhado entre os diferentes processos
m = SharedArray(Float64,1000,1000);
typeof(m)

SharedArray{Float64,2}

### @parallel

Feito unicamente para o for, além de o tornar paralelo ainda aceita funções quase como um *reduce* aprimorado

In [4]:
@time @parallel (+) for i = 1:100000
    i
end

  2.595335 seconds (423.78 k allocations: 18.048 MB, 0.60% gc time)


5000050000

In [5]:
@time @parallel for i = 1:length(m)
    m[i] = rand()
end

  0.522281 seconds (416.30 k allocations: 18.375 MB, 3.44% gc time)


4-element Array{Future,1}:
 Future(2,1,18,#NULL)
 Future(3,1,19,#NULL)
 Future(4,1,20,#NULL)
 Future(5,1,21,#NULL)

### @async

In [6]:
task = @time @async @parallel for i = 1:length(m)
    m[i] = rand()
end

  0.004703 seconds (203 allocations: 11.906 KB)


Task (done) @0x00007f2123e76e60

Só para comparar:

In [7]:
@time for i = 1:length(m)
    m[i] = rand()
end

  0.226384 seconds (4.00 M allocations: 76.369 MB, 11.14% gc time)


### Breve explicação do funcionamento do que está por trás disso:

De fato as macros são fundamentais para que o processo seja tão simples e a LLVM ajuda bastante na eficiência deste tipo de operação, mas manualmente poderíamos fazer tentar dividir o processamento usando as seguintes funções e macros:

* @spawn - encapsula uma expressão e a envia a outro processo, devolve um tipo `Futuro`
* fetch - executa um procedimento alocado em algum processo
* remotecall - chama uma função de forma asíncrona em determinado processo 

In [8]:
future0 = remotecall(rand, 4, 3) # funcao, processo, argumentos...
println("future0 = $future0")

future1 = @spawn 10 .+ fetch(future0)
println("future1 = $future1")

future2 = @spawnat 3 100 .+ fetch(future1) 
println("future2 = $future2")

fetch(future2)

future0 = Future(4,1,26,Nullable{Any}())
future1 = Future(2,1,27,Nullable{Any}())
future2 = Future(3,1,28,Nullable{Any}())


3-element Array{Float64,1}:
 110.5  
 110.014
 110.993

Obs¹.: a macro `@spawn` envia um procedimento para a thread mais livre do processador, mas podemos direcionar um procedimento para uma thread específica usando `@spawnat`

Obs².: `remotecall_fetch` é o mesmo que `fetch(remotecall(func, p, args...))`

### @everywhare

Permite que uma função possa estar disponível todos os processos da execução atual

In [9]:
function testeRand(dims...)
    return 2*rand(dims...)
end

@everywhere function testeRandEvery(dims...)
    return 2*rand(dims...)
end

In [10]:
@time testeRand(2)
@time testeRandEvery(2)

  0.113052 seconds (42.38 k allocations: 1.842 MB)
  0.003522 seconds (236 allocations: 13.438 KB)


2-element Array{Float64,1}:
 1.74152 
 0.705784

In [11]:
future0 = @spawn testeRandEvery(2)
println(future0)

future1 = @spawn testeRand(2)
println(future1)

@time fetch(future0)
@time fetch(future1)

Future(3,1,34,Nullable{Any}())
Future(4,1,35,Nullable{Any}())
  0.000616 seconds (101 allocations: 4.188 KB)


LoadError: On worker 4:
UndefVarError: #testeRand not defined
 in deserialize_datatype at ./serialize.jl:825
 in handle_deserialize at ./serialize.jl:571
 in collect at ./array.jl:307
 in deserialize at ./serialize.jl:588
 in handle_deserialize at ./serialize.jl:581
 in deserialize at ./serialize.jl:541
 in deserialize_datatype at ./serialize.jl:831
 in handle_deserialize at ./serialize.jl:571
 in deserialize_msg at ./multi.jl:120
 in message_handler_loop at ./multi.jl:1317
 in process_tcp_streams at ./multi.jl:1276
 in #638 at ./event.jl:68